In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Importing libraries


In [ ]:
import cv2
import numpy as np
import os

#Loading Templates


In [ ]:
def load_templates(templates_path):
    templates = {}
    for emotion_folder in os.listdir(templates_path):
        emotion_path = os.path.join(templates_path, emotion_folder)
        templates[emotion_folder] = []
        for template_name in os.listdir(emotion_path):
            template_path = os.path.join(emotion_path, template_name)
            template_img = cv2.imread(template_path, 0)
            templates[emotion_folder].append(template_img)
    return templates

#Matching the image with templates


In [ ]:
def custom_match_template(image, template):
    image_h, image_w = image.shape
    template_h, template_w = template.shape
    best_score = -1
    best_location = None
    template = template - np.mean(template)
    for y in range(image_h - template_h + 1):
        for x in range(image_w - template_w + 1):
            roi = image[y:y + template_h, x:x + template_w]
            roi = roi - np.mean(roi)
            numerator = np.sum(roi * template)
            denominator = np.sqrt(np.sum(roi**2) * np.sum(template**2))
            score = numerator / denominator if denominator != 0 else 0

            if score > best_score:
                best_score = score
                best_location = (x, y)

    return best_score, best_location

In [ ]:
def match_template(image, templates):
    best_score = 0
    best_emotion = None
    for emotion, template_list in templates.items():
        for template in template_list:
            score, _ = custom_match_template(image, template)

            if score > best_score:
                best_score = score
                best_emotion = emotion

    return best_emotion, best_score

#Load image to detect

In [ ]:
def load_image(image_path):
    image = cv2.imread(image_path, 0)
    if image is None:
        raise Exception(f"Failed to load image from {image_path}.")
    return image

#Driver

In [ ]:
def detect_emotion_from_image(image_path, templates):
    image = load_image(image_path)
    emotion, score = match_template(image, templates)
    print(f"Detected Emotion: {emotion}, Score: {score}")
    return emotion, score

In [ ]:
def main():
    templates_path = r"/content/drive/MyDrive/train"
    image_path = r"/content/drive/MyDrive/train/happy/im0.png"
    templates = load_templates(templates_path)
    detect_emotion_from_image(image_path, templates)


In [ ]:
if __name__ == "__main__":
    main()

Detected Emotion: happy, Score: 1.0
